https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da

In [ ]:
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install --quiet accelerate==0.21.0
!pip install --quiet peft==0.4.0
!pip install --quiet bitsandbytes==0.40.2
!pip install --quiet transformers==4.31.0
!pip install --quiet trl==0.4.7
!pip install --quiet wandb
!pip install --quiet evaluate
!pip install --quiet xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import time
import numpy as np

hyperparams

In [ ]:
new_model = "llama-2-7b-sentiment-classifier"
device_map = {"": 0}

dataset

In [ ]:
from datasets import load_dataset

imdb = load_dataset("imdb")
imdb = imdb.shuffle(seed=42)
dataset = imdb['train'].train_test_split(test_size=0.01, shuffle=False)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 24750
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 250
    })
})

format the prompt

In [ ]:
def format_dolly(sample):
    sys_prompt = """<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>"""
    instruction = f"### Instruction\nRate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1."
    context = f"### Context\n{sample['text']}"
    response = f"### Answer\n"
    prompt = "\n\n".join([i for i in [sys_prompt, instruction, context, response] if i is not None])
    return prompt

from random import randrange
print(format_dolly(dataset["train"][randrange(len(dataset["train"]))]))

<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>

### Instruction
Rate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1.

### Context
This was the first "Walking Tall" movie I saw, I think in a $2 movie theater along Hollywood Blvd. , so I didn't have any reference to the first installment done by Joe Don Baker. I remember being shocked at the corrupted system of McNairy County and the brutality of the "redneck gangs". I was also amazed at the fact that one man decided he's not going to let it slide,

util for getting predictions

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(f"foobar8675/{new_model}", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [ ]:
def parse_output(result, duration):
    answer = result[0]['generated_text']
    answer = answer.split("### Answer\n")[1]
    answer = answer.replace(".", "")

    print("---")
    print(result)
    print("---")
    print(answer)
    print("---")

    if "###" in answer:
        answer, explanation = answer.split("###")
        print(f"Explanation {explanation}")

    if len(answer) > 0:
        answer = answer[0].strip()

    if not answer.isdigit():
        print(f"HAD NO DIGIT\n\n{result}")
        answer = None

    return answer, round((duration),2)

def get_prediction(pipe, text):
    start = time.time()
    prompt = format_dolly({"text": text, "label": ""})
    result = pipe(
        f"{prompt}",
        do_sample=True,
        top_k=1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
        temperature=0.1
    )
    end = time.time()
    return parse_output(result, end-start)

def get_predictions(pipe, ds, n):
    ret = []
    for i in range(n):
        answer, timing = get_prediction(pipe, ds[[i]]["text"])
        ret.append(answer)
        if answer:
            print(f'label:{dataset["test"][[i]]["label"]} prediction:{answer} timing: {round(timing, 2)} s')
        else:
            print(f'no valid answer for {ds[[i]]} timing: {round(timing, 2)} s')
    return ret

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    f"foobar8675/{new_model}",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
ds = dataset['test'].select(range(50))

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

refs = []
preds = []
skipped = []
timings = []
iterable_dataset = ds.to_iterable_dataset()
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024, device_map=device_map)
for i, sample in enumerate(iterable_dataset):
    answer, timing = get_prediction(pipe, sample['text'])
    if answer is None:
        skipped.append(i)
        print("skipped")
        answer = -10
    else:
        preds.append(int(answer))
        refs.append(int(sample['label']))
        timings.append(timing)

accuracy.add_batch(references=refs, predictions=preds)
accuracy.compute()

---
[{'generated_text': '<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\n    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information.<</SYS>>\n\n### Instruction\nRate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1.\n\n### Context\nThere\'s no shortage of bad dialogue in David and Bathsheba \x96 "I was quite a hand with a slingshot," "The King of all Israel out there in the darkness exposing himself to the enemy" (full marks to Dennis Hooey for delivering that one with a straight face), "Go and sit with the concubines." And somehow I doubt a bored David ever tol

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Input length of input_ids is 1238, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


---
[{'generated_text': "<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\n    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n\n### Instruction\nRate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1.\n\n### Context\nThis is a great game! Okay perhaps it didn't have some of the features it was meant to have but Digital Anvil have still come up with a good game. There is a certain similarity to Elite(you can trade, pick up weapons and cargo off destroyed ships, go on missions)however this game features a heavily scripted mission. It is a great missio

Input length of input_ids is 1386, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


---
[{'generated_text': "<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\n    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n\n### Instruction\nRate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1.\n\n### Context\nWhat a disappointment, especially in light of the budget provided, the technical resources available, and the talent assembled. Isn't the fundamental rule for science fiction/drama to create in the audience the willing suspension of disbelief. POA 2001 creates a plausible beginning, suckering us in, but thereinafter Mr. Burton forgets t

Input length of input_ids is 1363, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


---
[{'generated_text': "<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\n    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n\n### Instruction\nRate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1.\n\n### Context\nNo Fireworks Despite Violent Action.<br /><br />Science fiction films that reflect quality are scarce indeed, largely because transposal of imaginative themes from the genre to the screen too often falls short of effective execution as a result of insufficient funding or inadequate invention, and unfortunately for its producers, this wo

{'accuracy': 0.96}

In [ ]:
f"{round(np.average(timings), 2)} seconds/report"

'16.11 seconds/report'

In [ ]:
from transformers import pipeline
import evaluate
accuracy = evaluate.load("accuracy")
refs = []
preds = []
durations = []
iterable_dataset = ds.to_iterable_dataset()
for sample in iterable_dataset:
    start = time.time()
    classifier = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english", truncation=True)
    duration = time.time()-start
    result = classifier(sample["text"])
    preds.append(int(result[0]['label'] == 'POSITIVE'))
    refs.append(int(sample['label']))
    durations.append(duration)

accuracy.add_batch(references=refs, predictions=preds)
print(f"{accuracy.compute()}, {round(np.average(durations),2)}s")

{'accuracy': 0.96}, 3.63s


In [ ]:
print(f"{round(np.average(durations),2)} seconds per report")

3.63 seconds per report
